# Lab 2: Train Hugging Face Transformers on Amazon SageMaker

### Korean NLP Downstream task: Question Answering
---


## Introduction
---

바로 이전 모듈에서 기존에 온프레미스에서 개발했던 환경과 동일한 환경으로 모델을 빌드하고 훈련했습니다. 하지만 아래와 같은 상황들에서도 기존 환경을 사용하는 것이 바람직할까요?

- 온프레미스의 GPU는 총 1장으로 훈련 시간이 너무 오래 소요됨
- 가용 서버 대수가 2대인데 10개의 딥러닝 모델을 동시에 훈련해야 함
- 필요한 상황에만 GPU를 활용
- 기타 등등

Amazon SageMaker는 데이터 과학자들 및 머신 러닝 엔지니어들을 위한 완전 관리형 머신 러닝 서비스로 훈련 및 추론 수행 시 인프라 설정에 대한 추가 작업이 필요하지 않기에, 단일 GPU 기반의 딥러닝 훈련을 포함한 멀티 GPU 및 멀티 인스턴스 분산 훈련을 보다 쉽고 빠르게 수행할 수 있습니다. SageMaker는 다양한 유즈케이스들에 적합한 예제들을 지속적으로 업데이트하고 있으며, 한국어 세션 및 자료들도 제공되고 있습니다.

### Notes

이미 기본적인 Hugging Face 용법 및 자연어 처리에 익숙하신 분들은 앞 모듈을 생략하고 이 모듈부터 핸즈온을 시작하셔도 됩니다.
이 노트북은 SageMaker 기본 API를 참조하므로, SageMaker Studio, SageMaker 노트북 인스턴스 또는 AWS CLI가 설정된 로컬 시스템에서 실행해야 합니다. SageMaker Studio 또는 SageMaker 노트북 인스턴스를 사용하는 경우 PyTorch 기반 커널을 선택하세요.
훈련 job 수행 시 최소 `ml.g4dn.xlarge` 이상의 훈련 인스턴스가 필요하며, `ml.p3.8xlarge`나 `ml.p3.16xlarge` 인스턴스를 권장합니다. 만약 인스턴스 사용에 제한이 걸려 있다면 Request a service quota increase for SageMaker resources를 참조하여 인스턴스 제한을 해제해 주세요.

### References
- Hugging Face Tutorial: https://huggingface.co/docs/transformers/training
- 네이버, 창원대가 함께하는 NLP Challenge GitHub: https://github.com/naver/nlp-challenge
- 네이버, 창원대가 함께하는 NLP Challenge 리더보드 및 라이센스: http://air.changwon.ac.kr/?page_id=10


## 1. Setup Environments
---

### Import modules

In [1]:
import boto3
import sagemaker
import sagemaker.huggingface

sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::143656149352:role/service-role/AmazonSageMaker-ExecutionRole-20220317T150353
sagemaker bucket: sagemaker-us-east-1-143656149352
sagemaker session region: us-east-1


In [2]:
import os
import sys
import logging
import argparse
import torch
from torch import nn
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertTokenizerFast, BertConfig, BertForTokenClassification

logging.basicConfig(
    level=logging.INFO, 
    format='[{%(filename)s:%(lineno)d} %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger(__name__)

### Load Feature set

In [3]:
train_dir = 'qna_train'
valid_dir = 'qna_valid'

!wget https://korquad.github.io/dataset/KorQuAD_v1.0_train.json -O {train_dir}/KorQuAD_v1.0_train.json
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json -O {valid_dir}/KorQuAD_v1.0_dev.json

--2022-07-06 04:57:21--  https://korquad.github.io/dataset/KorQuAD_v1.0_train.json
Resolving korquad.github.io (korquad.github.io)... 185.199.108.153, 185.199.111.153, 185.199.109.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38527475 (37M) [application/json]
Saving to: ‘qna_train/KorQuAD_v1.0_train.json’

100%[======================================>] 38,527,475  --.-K/s   in 0.1s    

2022-07-06 04:57:21 (269 MB/s) - ‘qna_train/KorQuAD_v1.0_train.json’ saved [38527475/38527475]

--2022-07-06 04:57:21--  https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json
Resolving korquad.github.io (korquad.github.io)... 185.199.110.153, 185.199.109.153, 185.199.111.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3881058 (3.7M) [application/json]
Saving to: ‘qna_valid/KorQuAD_v1.0_dev.json’

1

### Uploading data to Amazon S3 Bucket

SageMaker 훈련을 위해 데이터셋을 S3로 업로드합니다.

In [4]:
bucket = sess.default_bucket()

# s3 key prefix for the data
s3_prefix = 'samples/datasets/korquad'

# save train_dataset to s3
train_input_path = f's3://{bucket}/{s3_prefix}/train'
valid_input_path = f's3://{bucket}/{s3_prefix}/valid'

In [5]:
%%time
!aws s3 cp {train_dir} {train_input_path} --recursive --quiet
!aws s3 cp {valid_dir} {valid_input_path} --recursive --quiet

CPU times: user 7.29 ms, sys: 42 ms, total: 49.3 ms
Wall time: 1.74 s


<br>

## 2. Training with Native Hugging Face (PyTorch Framework)

---

### Overview and Training Script

SageMaker에 대한 대표적인 오해가 여전히 많은 분들이 SageMaker 훈련을 위해 소스 코드를 전면적으로 수정해야 한다고 생각합니다. 하지만, 실제로는 별도의 소스 코드 수정 없이 기존 여러분이 사용했던 파이썬 스크립트에 SageMaker 훈련에 필요한 SageMaker 전용 환경 변수들만 추가하면 됩니다. 

SageMaker 훈련은 훈련 작업을 호출할 때, 1) 훈련 EC2 인스턴스 프로비저닝 - 2) 컨테이너 구동을 위한 도커 이미지 및 훈련 데이터 다운로드 - 3) 컨테이너 구동 - 4) 컨테이너 환경에서 훈련 수행 - 5) 컨테이너 환경에서 S3의 특정 버킷에 저장 - 6) 훈련 인스턴스 종료로 구성됩니다. 따라서, 훈련 수행 로직은 아래 예시와 같이 기존 개발 환경과 동일합니다.

```python
/opt/conda/bin/python train.py --epochs 5 --train_batch_size 32 ...
```

이 과정에서 컨테이너 환경에 필요한 환경 변수(예: 모델 경로, 훈련 데이터 경로) 들은 사전에 지정되어 있으며, 이 환경 변수들이 설정되어 있어야 훈련에 필요한 파일들의 경로를 인식할 수 있습니다. 대표적인 환경 변수들에 대한 자세한 내용은 https://github.com/aws/sagemaker-containers#important-environment-variables 을 참조하세요.

In [6]:
from sagemaker.huggingface import HuggingFace
import time
instance_type = 'ml.p3.8xlarge'
num_gpus = 4
instance_count = 1
train_batch_size = 16
eval_batch_size = 64
max_length = 384
stride = 64
model_id = 'salti/bert-base-multilingual-cased-finetuned-squad' 
fp16 = True
tokenizer_id = model_id
logging_steps = 100
learning_rate = 5e-5
logger.info(f'learning_rate: {learning_rate}')

# hyperparameters, which are passed into the training job
hyperparameters = {
    'n_gpus': num_gpus,                   # number of GPUs per instance
    'epochs': 3,                          # number of training epochs
    'seed': 42,                           # random seed
    'train_batch_size': train_batch_size, # batch size for training
    'eval_batch_size': eval_batch_size,   # batch size for evaluation
    'max_length': max_length,             # max sequence
    'stride': stride,                     # stride
    'logging_steps': logging_steps,       # logging steps
    'learning_rate': learning_rate,       # learning rate used during training
    'fp16': fp16,                         # use FP16
    'tokenizer_id': tokenizer_id,         # pre-trained tokenizer
    'model_id': model_id                  # pre-trained model
}

[{2571691105.py:15} INFO - learning_rate: 5e-05


In [7]:
# define Training Job Name 
job_name = f'kornlp-qna-training-{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'
chkpt_s3_path = f's3://{bucket}/{s3_prefix}/native/checkpoints'

# create the Estimator
sm_estimator = HuggingFace(
    entry_point           = 'train.py',         # fine-tuning script used in training jon
    source_dir            = './scripts',        # directory where fine-tuning script is stored
    instance_type         = instance_type,      # instances type used for the training job
    instance_count        = instance_count,     # the number of instances used for training
    base_job_name         = job_name,           # the name of the training job
    role                  = role,               # IAM role used in training job to access AWS ressources, e.g. S3
    transformers_version  = '4.17.0',           # the transformers version used in the training job
    pytorch_version       = '1.10.2',           # the pytorch_version version used in the training job
    py_version            = 'py38',             # the python version used in the training job
    hyperparameters       = hyperparameters,    # the hyperparameter used for running the training job
    disable_profiler      = True,
    debugger_hook_config  = False, 
    checkpoint_s3_uri     = chkpt_s3_path,
    checkpoint_local_path ='/opt/ml/checkpoints',  
)

logger.info(f'training job name: {job_name}')

[{581002583.py:23} INFO - training job name: kornlp-qna-training-2022-07-06-04-57-30


`fit()` 메소드를 호출하여 훈련 job을 시작합니다. `fit()` 메소드의 인자값 중 `wait=True`로 설정할 경우에는 동기(synchronous) 방식으로 동직하게 되며, `wait=False`일 경우에는 비동기(aynchronous) 방식으로 동작하여 여러 개의 훈련 job을 동시에 실행할 수 있습니다.

In [8]:
# define a data input dictonary with our uploaded s3 uris
data = {
    'train': train_input_path,
    'valid': valid_input_path
}

# starting the train job with our uploaded datasets as input
sm_estimator.fit(data, wait=False)
train_job_name = sm_estimator.latest_training_job.job_name

[{session.py:590} INFO - Creating training-job with name: kornlp-qna-training-2022-07-06-04-57-30-2022-07-06-04-57-31-762


### View Training Job
SageMaker 콘솔 창에서 훈련 내역을 직접 확인할 수도 있지만, 아래 코드 셀에서 생성되는 링크를 클릭하면 더 편리하게 훈련 내역을 확인할 수 있습니다.

In [9]:
from IPython.core.display import display, HTML

def make_console_link(region, train_job_name, train_task='[Training]'):
    train_job_link = f'<b> {train_task} Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={region}#/jobs/{train_job_name}">Training Job</a></b>'   
    cloudwatch_link = f'<b> {train_task} Review <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={region}#logStream:group=/aws/sagemaker/TrainingJobs;prefix={train_job_name};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a></b>'
    return train_job_link, cloudwatch_link  
        
train_job_link, cloudwatch_link = make_console_link(region, train_job_name, '[Hugging Face Training - Native]')

display(HTML(train_job_link))
display(HTML(cloudwatch_link))

### Wait for the training jobs to complete
훈련이 완료될 때까지 기다립니다. `estimator.fit(...)`에서 `wait=False`로 설정한 경우, 아래 코드 셀의 주석을 해제 후 실행하여 동기 방식으로 변경할 수도 있습니다. 훈련 완료까지는 약 15-20분의 시간이 소요됩니다.

In [10]:
sess.logs_for_job(job_name=train_job_name, wait=True)

2022-07-06 05:01:19 Starting - Starting the training job......
2022-07-06 05:01:58 Starting - Preparing the instances for training.........
2022-07-06 05:03:25 Downloading - Downloading input data......
2022-07-06 05:04:36 Training - Downloading the training image.......................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
/opt/conda/lib/python3.8/site-packages/paramiko/transport.py:236: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,
2022-07-06 05:08:30,777 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-07-06 05:08:30,816 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-07-06 05:08:30,824 sagemaker_pytorch_container.training INFO     Invoking user training script.

2022-07-06 05:08:27 Training - Training image download completed. Training in progress.2022-07-06 05:09:55,362 s

### Copy model artifacts from S3 to local path

훈련된 모델 파라메터는 `model.tar.gz`로 압축되어 S3에 저장됩니다. 만약 SageMaker 상에서 훈련한 모델을 곧바로 배포한다면, 아래 코드 셀을 실행할 필요는 없지만, 로컬/개발 환경에서 훈련된 모델을 간단히 테스트하거나 다른 환경에서 모델을 서빙할 때는 S3에 저장된 모델을 다운로드하셔야 합니다.

In [11]:
local_model_dir = './model'
!rm -rf {local_model_dir}
s3_model_path = sm_estimator.model_data
os.makedirs(local_model_dir, exist_ok=True)

In [12]:
%%bash -s "$local_model_dir" "$s3_model_path"
aws s3 cp $2 $1
cd $1
tar -xzvf model.tar.gz

download: s3://sagemaker-us-east-1-143656149352/kornlp-qna-training-2022-07-06-04-57-30-2022-07-06-04-57-31-762/output/model.tar.gz to model/model.tar.gz
vocab.txt
training_args.bin
tokenizer_config.json
tokenizer.json
special_tokens_map.json
pytorch_model.bin
config.json


In [13]:
%store s3_model_path local_model_dir model_id tokenizer_id

Stored 's3_model_path' (str)
Stored 'local_model_dir' (str)
Stored 'model_id' (str)
Stored 'tokenizer_id' (str)


<br>

## 3. Prediction
---
### Load fine-tuned model

In [14]:
from transformers import BertForQuestionAnswering
tokenizer = BertTokenizerFast.from_pretrained(f'{local_model_dir}')
model = BertForQuestionAnswering.from_pretrained(f'{local_model_dir}')
model.load_state_dict(torch.load(f'{local_model_dir}/pytorch_model.bin'))
model = model.eval()

In [15]:
def predict_fn(example, model):
    
    from transformers import pipeline
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    device_id = -1 if device.type == "cpu" else 0
        
    context = example[0]
    question = example[1]
    
    nlp = pipeline("question-answering", model=model.to(device_id), device=device_id,
                   tokenizer=tokenizer)
    results = nlp(question=question, context=context)
    return results

### Example
여러분만의 샘플 문장을 만들어서 자유롭게 추론을 수행해 보세요.

In [16]:
context = r"""
아마존웹서비스(AWS)는 카카오 게임 전문 계열사 카카오게임즈가 자사 머신러닝(ML), 데이터베이스(DB) 및 데이터 분석 등 서비스를 통해 사용자 경험을 제고했다고 7일 밝혔다.
AWS는 카카오게임즈가 AWS클라우드 역량을 활용해 게임 데이터 분석 솔루션을 실행하고, 대량의 게임 데이터와 설치 건수, 사용자 유지율과 같은 성과 지표를 분석하고 있다고 설명했다. 
현재 카카오게임즈는 폭증하는 데이터를 저장·분석하기 위한 방법으로 클라우드 오브젝트 스토리지 서비스 '아마존 S3(Amazon Simple Storage Service)' 기반 데이터 레이크(Data Lake)를 구축했다. 또 데이터 분석을 용이하게 해주는 대화형 쿼리 서비스 '아마존 아테나(Amazon Athena)'를 도입해 데이터 레이크로부터 게임 데이터를 통합하고, 게임 사용자 행동과 관련된 인사이트를 확보 중이다. 
이를 통해 카카오게임즈는 게임 봇을 탐지하고 제거하는 방식으로 사용자 경험을 제고했다. 또한 관계형 데이터베이스 서비스 '아마존 오로라(Amazon Aurora)'를 활용해 게임 내 구매와 같은 대규모 데이터베이스 거래를 처리하고 있다. 이밖에도 카카오게임즈는 ML 모델 구축, 교육 및 배포를 위한 완전 관리형 서비스 '아마존 세이지메이커(Amazon SageMaker)'를 활용할 예정이다.
"""

In [17]:
question = "카카오 게임 전문 계열사는?"
print(predict_fn((context, question), model))

question = "AWS의 클라우드 오브젝트 스토리지 서비스는?"
print(predict_fn((context, question), model))

question = "AWS의 관계형 데이터베이스 서비스는?"
print(predict_fn((context, question), model))

question = "AWS의 ML 모델 완전 관리형 서비스는?"
print(predict_fn((context, question), model))

{'score': 0.998553991317749, 'start': 29, 'end': 36, 'answer': '카카오게임즈가'}
{'score': 0.8284631967544556, 'start': 263, 'end': 269, 'answer': '아마존 S3'}
{'score': 0.9286826252937317, 'start': 514, 'end': 521, 'answer': '아마존 오로라'}
{'score': 0.8117018938064575, 'start': 626, 'end': 636, 'answer': '아마존 세이지메이커'}
